In [632]:
import numpy as np
from math import sqrt, ceil, pi, sin, cos, tan, asin, acos, atan, log
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=2, suppress=True)
%precision 2


P = 7 # kW
n = 65 # rpm
L = 8 # years
Kng = 260 # days
Ca = 1 # shifts
t1 = 15 # s
t2 = 11 # s
T1 = 1 # N.mm
T2 = .7 # N.mm

def odd(num):
    if num%2==0: return num+1
    else: return num
def disp(*args):
    a = []
    for arg in args:
        a.append(arg)
    return np.array(a,dtype=float)

fkses1 = interp1d([400,500,600,700,800,900,1000,1200],
            [2.25,2.5,2.75,3,3.25,3.5,3.75,4.25])
fkses2 = interp1d([400,500,600,700,800,900,1000,1200],
             [2.75,3.05,3.36,3.66,3.96,4.28,4.6,5.2])
fkses1 = interp1d([400,500,600,700,800,900,1000,1200],
            [2.25,2.5,2.75,3,3.25,3.5,3.75,4.25])
fkses2 = interp1d([400,500,600,700,800,900,1000,1200],
             [2.75,3.05,3.36,3.66,3.96,4.28,4.6,5.2])
fes = interp1d([15,20,30,40,50,70,80,100],
              [0.87,0.83,0.77,0.73,0.7,0.66,0.64,0.62])
fet = interp1d([15,20,30,40,50,70,80,100],
               [0.92,0.89,0.81,0.78,0.76,0.73,0.71,0.7])

# fes(850),fes(850)

## Motor Design $(power, rpm)$

In [633]:
# calculate efficiency, e = eta
e_c = 1
e_b = 0.99
e_hg = 0.97 # helical gear
e_ch = 0.96
e_sys = e_c * e_b**4 * e_hg**2 * e_ch
e_sys

0.87

In [634]:
P_max = P # calculate max power of the belt conveyor, kW
P_w = P_max*sqrt((T1**2*t1 + T2**2*t2)/(t1+t2)) # calculate power given the work load, kW
P_mo = P_w/e_sys # lower limit of the motor's power, kW
Kqt = sqrt((T1**2*t1 + T2**2*t2)/(t1+t2))**-1
P_w, P_mo, Kqt # compare P_max & P_mo, choose larger value to set lower limit and look up the catalogue

(6.20, 7.14, 1.13)

In [635]:
u_ch = 2.8
u_h = 8
u_sys = u_ch * u_h # calculate transmission ratio of the system
n_mo = u_sys * n # calculate theoretical rotational speed of the motor, rpm
n, u_sys, n_mo

(65, 22.40, 1456.00)

In [636]:
P_mo = 7.5 # power based on choice of motor, kW
n_mo = 1455 # rotational speed based on choice of motor, rpm
u_sys = n_mo / n # new transmission ratio
u_ch = u_sys / u_h # new transmission ratio of ch drive, given u_hg=const
u_sys, u_ch

(22.38, 2.80)

In [637]:
P_ch  = P_w / e_b
P_sh3 = P_ch / e_ch # power of shaft 3, kW
P_sh2 = P_sh3 / (e_b*e_hg)
P_sh1 = P_sh2 / (e_b*e_hg)
P_mow = P_sh1 / e_b
P_mow, P_sh1, P_sh2, P_sh3, P_ch

(7.14, 7.07, 6.79, 6.52, 6.26)

In [638]:
u_1 = sqrt(u_h)
u_2 = u_h/u_1
n_sh1 = n_mo # rotational speed of shaft 1, rpm
n_sh2 = n_sh1 / u_1
n_sh3 = n_sh2 / u_2
u_1, u_2, n_sh1, n_sh2, n_sh3

(2.83, 2.83, 1455, 514.42, 181.88)

In [639]:
T_mo  = 9.55e6 * P_mow / n_mo # torque of motor, N.mm
T_sh1 = 9.55e6 * P_sh1 / n_sh1 # torque of shaft 1, N.mm
T_sh2 = 9.55e6 * P_sh2 / n_sh2
T_sh3 = 9.55e6 * P_sh3 / n_sh3
T_mo, T_sh1, T_sh2, T_sh3

(46892.66, 46423.73, 126093.30, 342486.86)

## Chain Drive Design

In [640]:
# find z1, z2
n1 = n_sh3
z1 = 29 - 2*u_ch
n1, z1

(181.88, 23.40)

In [641]:
z1 = odd(ceil(z1))
z2 = u_ch*z1 # eq (5.1), p.80, obtain odd number of teeth
z1, z2

(25, 69.95)

In [642]:
kz, kn, k0, ka, kdc, kd, kc, kbt = 25/z1, 200/n1, 1, 1, 1, 1.5, 1, 1.3
Pt = P_ch*k0*ka*kdc*kd*kc*kz*kn
kz,kn,Pt

(1.00, 1.10, 10.33)

In [643]:
# find P_allowance and [a_min, a_max]
# P_allowance, p, dc, B = 19.3, 31.75, 9.55, 27.46 # table 5.5, p.81
z2 = odd(ceil(z2)) # eq (5.1), p.80, obtain odd number of teeth
P_allowance, p, dc, B = 11, 25.4, 7.95, 22.61 # table 5.5, p.81
a_min, a_max = 30*p, 50*p
z2, a_min, a_max

(71, 762.00, 1270.00)

In [644]:
a = a_min # mm
x = 2*a/p + (z1+z2)/2 + (z2-z1)**2*p/(4*pi**2*a) # eq (5.12), p.85
xc = odd(ceil(x)) + 1
a = p/4*(xc - 0.5*(z2+z1) + sqrt((xc-0.5*(z2+z1))**2-2*((z2-z1)/pi)**2)) * (1-.003)
i = z1*n1/(15*xc)
x, xc, a, i

(109.79, 112, 788.57, 2.71)

In [645]:
# k_d, dl, Q, q, k_f = 1.2, 19.05, 88.5, 3.8, 6 # table 5.2; eq 5.15
k_d, dl, Q, q, k_f = 1.2, 15.88, 56.7, 2.6, 6 # table 5.2; eq 5.15
fs = interp1d([50,200,400,600,800,1e3],[7,8.2,9.3,10.3,11.7,12.9]) # table 5.10
v1 = n1*p*z1/6e4
Ft, Fv, F0 = 1e3*P_ch/v1, q*v1**2, 9.81*k_f*q*a/1e3
s = Q*1e3/(k_d*Ft + F0 + Fv)
s_allowable = float(fs(n1))
v1, Ft, Fv, F0, s, s_allowable

(1.92, 3253.04, 9.63, 120.68, 14.06, 8.05)

In [646]:
sigHm, E1, E2, A, kd = 600, 205000, 117000, 180, 1
E = E1*E2*2/(E1+E2)
Fvd = 13e-7*n1*p**3
fkr = interp1d([15,20,30,40,50,60],[.59,.48,.36,.29,.24,.22])
k_r = float(fkr(z1))
sigH=.47*sqrt(k_r*(Ft*k_d+Fvd)*E/A)
Fvd, k_r, E, sigH, sigHm

(3.87, 0.42, 148975.16, 547.76, 600)

In [647]:
k_x = 1.15
F2 = F0+Fv
F1 = Ft+F2
Fr = Fr32 = k_x*Ft
F2, F1, Fr

(130.31, 3383.35, 3741.00)

In [648]:
d1 = p/sin(pi/z1)
d2 = p/sin(pi/z2)
da1 = p*(.5 + 1/tan(pi/z1))
da2 = p*(.5 + 1/tan(pi/z2))
df1 = d1 - 2*(.5025*dl+.05)
df2 = d2 - 2*(.5025*dl+.05)
d1, d2, da1, da2, df1, df2

(202.66, 574.23, 213.76, 586.37, 186.60, 558.17)

## Gear design

In [649]:
HB, sigb, sigch = 205, 600, 340 # MPa
SH, SF = 1.1, 1.75
H1, H2 = HB, HB-15
sigHlim1, sigFlim1 = 2*H1+70, 1.8*H1
sigHlim2, sigFlim2 = 2*H2+70, 1.8*H2
H1, H2, sigHlim1, sigFlim1, sigHlim2, sigFlim2

(205, 190, 480, 369.00, 450, 342.00)

In [650]:
mH, mF, c, NFO = 6, 6, 1, 4e6

NHO1, NHO2 = 30*H1**2.4, 30*H2**2.4
Lh = 8*Ca*Kng*L
tmp = 60*Lh*c*(T1**3*t1 + T2**3*t2)/(t1+t2)
NHE1q, NHE2q, NHE1s, NHE2s = n_sh1*tmp, n_sh2*tmp, n_sh2*tmp, n_sh3*tmp
tmp = 60*Lh*c*(T1**mF*t1 + T2**mF*t2)/(t1+t2)
NFE1q, NFE2q, NFE1s, NFE2s = n_sh1*tmp, n_sh2*tmp, n_sh2*tmp, n_sh3*tmp
NHO1, NHO2, Lh, np.array([NHE1q, NHE2q, NFE1q, NFE2q, NHE1s, NHE2s, NFE1s, NFE2s])/1e9

(10600601.81,
 8833440.68,
 16640,
 array([1.05, 0.37, 0.91, 0.32, 0.37, 0.13, 0.32, 0.11]))

In [651]:
fh = lambda nho, nhe: (nho/nhe)**(1/mH)
ff = lambda nfe: (NFO/nfe)**(1/mF)
KHL1q, KHL2q, KHL1s, KHL2s = fh(NHO1,NHE1q), fh(NHO2,NHE2q), fh(NHO1,NHE1s), fh(NHO2,NHE2s)
KFL1q, KFL2q, KFL1s, KFL2s = ff(NFE1q), ff(NFE2q), ff(NFE1s), ff(NFE2s)
KHL1q, KHL2q, KFL1q, KFL2q, KHL1s, KHL2s, KFL1s, KFL2s

(0.46, 0.54, 0.40, 0.48, 0.55, 0.64, 0.48, 0.57)

In [652]:
cond = lambda k: 1 if k < 1 else k
KFC = 1 # one direction rotation
KHL1q, KHL2q, KHL1s, KHL2s, KFL1q, KFL2q, KFL1s, KFL2s = cond(KHL1q), cond(KHL2q), cond(KHL1s), cond(KHL2s), cond(KFL1q), cond(KFL2q), cond(KFL1s), cond(KFL2s)
KHL1q, KHL2q, KHL1s, KHL2s, KFL1q, KFL2q, KFL1s, KFL2s

sigH1q, sigH1s = sigHlim1/SH*KHL1q, sigHlim1/SH*KHL1s
sigH2q, sigH2s = sigHlim2/SH*KHL2q, sigHlim2/SH*KHL2s
sigF1q, sigF1s = sigFlim1/SF*KFL1q*KFC, sigFlim1/SF*KFL1s*KFC
sigF2q, sigF2s = sigFlim2/SF*KFL2q*KFC, sigFlim2/SF*KFL2s*KFC
sigH1q, sigH2q, sigF1q, sigF2q, sigH1s, sigH2s, sigF1s, sigF2s

(436.36, 409.09, 210.86, 195.43, 436.36, 409.09, 210.86, 195.43)

In [653]:
avg = lambda a, b: (a+b)/2
sigH1, sigH2 = avg(sigH1q,sigH2q), avg(sigH1s,sigH2s)
sigF1, sigF2 = avg(sigF1q,sigF2q), avg(sigF1s,sigF2s)
sighmax, sigfmax = 2.8*sigch, 0.8*sigch
sigH1, sigH2, sigF1, sigF2, 1.25*min([sigH1q,sigH2q]), 1.25*min([sigH1s,sigH2s])

(422.73, 422.73, 203.14, 203.14, 511.36, 511.36)

In [654]:
sigH, sigF1, sigF2 = sigH1, sigF1q, sigF2q
Ka, psibaq, psibas, uq, us = 43, 0.3, 0.39, u_1, u_2
psibdq = 0.53*psibaq*(uq+1)
psibds = 0.53*psibas*(us+1)
fh5 = interp1d([.2,.4,.6,.8,1,1.2,1.4,1.6],[1.01,1.02,1.03,1.05,1.07,1.1 ,1.13,1.16])
fh4 = interp1d([.2,.4,.6,.8,1,1.2,1.4,1.6],[1.01,1.03,1.05,1.08,1.11,1.13,1.17,1.21])
ff5 = interp1d([.2,.4,.6,.8,1,1.2,1.4,1.6],[1.02,1.05,1.08,1.12,1.16,1.22,1.28,1.37])
ff4 = interp1d([.2,.4,.6,.8,1,1.2,1.4,1.6],[1.03,1.06,1.12,1.17,1.23,1.3 ,1.38,1.45])
KHbq, KHbs = float(fh5(psibdq)), float(fh4(psibds))
KFbq, KFbs = float(ff5(psibdq)), float(ff4(psibds))
psibdq, KHbq, KFbq, psibds, KHbs

(0.61, 1.03, 1.08, 0.79, 1.08)

In [655]:
# find center distance
awq = Ka*(uq+1)*(T_sh1*KHbq/sigH**2/uq/psibaq)**(1/3)
aws = Ka*(us+1)*(T_sh2*KHbs/sigH**2/us/psibas)**(1/3)
awq, aws

(112.08, 145.47)

In [656]:
awq = aws = 160
mq = np.array([.01,.02])*awq
ms = np.array([.01,.02])*aws
mq, ms

(array([1.6, 3.2]), array([1.6, 3.2]))

In [657]:
mq = 2; ms = 3; betq = 20; bets = 20
z1q = 2*awq*cos(betq*pi/180) / (mq*(uq+1))
z1s = 2*aws*cos(bets*pi/180) / (ms*(us+1))
z1q, z1s

(39.27, 26.18)

In [658]:
z1q, z1s = 41, 27
z2q, z2s = uq*z1q, us*z1s
bq, bs = psibaq*awq, psibas*aws; bs = 62
z2q, z2s, bq, bs

(115.97, 76.37, 48.00, 62)

In [659]:
z2q, z2s = 116, 77
betq, bets = acos(mq*(z1q+z2q)/2/awq), acos(ms*(z1s+z2s)/2/aws)
np.array([betq, bets])*180/pi

array([11.11, 12.84])

In [660]:
alpn = 20*pi/180
d1q = mq*z1q/cos(betq)
d2q = mq*z2q/cos(betq)
da1q = d1q+2*mq
da2q = d2q+2*mq
df1q = d1q-2.5*mq
df2q = d2q-2.5*mq
db1q = d1q*cos(alpn)
db2q = d2q*cos(alpn)
alptq = atan(tan(alpn)/cos(betq))
ealpq = (sqrt(da1q**2-db1q**2)+sqrt(da2q**2-db2q**2)-2*awq*sin(alptq))/(2*pi*mq*cos(alptq)/cos(betq))
v1q = pi*d1q*n_sh1/6e4
psibdq = bq/d1q
d1q, d2q, da1q, da2q, db1q, db2q, df1q, df2q, alptq*180/pi, v1q

(83.57, 236.43, 87.57, 240.43, 78.53, 222.17, 78.57, 231.43, 20.35, 6.37)

In [661]:
d1s = ms*z1s/cos(bets)
d2s = ms*z2s/cos(bets)
da1s = d1s+2*ms
da2s = d2s+2*ms
df1s = d1s-2.5*ms
df2s = d2s-2.5*ms
db1s = d1s*cos(alpn)
db2s = d2s*cos(alpn)
alpts = atan(tan(alpn)/cos(bets))
ealps = (sqrt(da1s**2-db1s**2)+sqrt(da2s**2-db2s**2)-2*aws*sin(alpts))/(2*pi*ms*cos(alpts)/cos(bets))
v1s = pi*d1s*n_sh2/6e4
psibds = bs/d1s
d1s, d2s, da1s, da2s, db1s, db2s, df1s, df2s, alpts*180/pi, v1s

(83.08, 236.92, 89.08, 242.92, 78.07, 222.63, 75.58, 229.42, 20.47, 2.24)

In [662]:
zr, kxh, yr, kxf = 1, 1, 1, 1
zvq = 0.85*v1q**.1
zvs = 0.85*v1s**.1
ysq = 1.08-0.0695*log(mq)
yss = 1.08-0.0695*log(ms)
sighq = sigH*zr*zvq*kxh
sigf1q = sigF1*yr*ysq*kxf
sigf2q = sigF2*yr*ysq*kxf
sighs = sigH*zr*zvs*kxh
sigf1s = sigF1*yr*yss*kxf
sigf2s = sigF2*yr*yss*kxf
zvq, zvs, ysq, yss, sighq, sigf1q, sigf2q, sighs, sigf1s, sigf2s

(1.02, 0.92, 1.03, 1.00, 432.38, 217.57, 201.65, 389.46, 211.63, 196.14)

In [663]:
zm = 274
betbq = atan(cos(alptq)*tan(betq))
betbs = atan(cos(alpts)*tan(bets))
zhq = sqrt(2*cos(betbq)/sin(2*alptq))
zhs = sqrt(2*cos(betbs)/sin(2*alpts))
betbq*180/pi, betbs*180/pi, zhq, zhs

(10.43, 12.05, 1.74, 1.73)

In [664]:
ebetq = bq*sin(betq)/mq/pi
if ebetq==0:
    zeq = sqrt((4-ealpq)/3)
elif ebetq < 1:
    zeq = sqrt((4-ealpq)*(1-ebetq)/3+ebetq/ealpq)
else: zeq = sqrt(1/ealpq)
ebets = bs*sin(bets)/ms/pi
if ebets==0:
    zes = sqrt((4-ealps)/3)
elif ebets < 1:
    zes = sqrt((4-ealps)*(1-ebets)/3+ebets/ealps)
else: zes = sqrt(1/ealps)
ealpq, zeq, ealps, zes, ebetq, ebets

(1.61, 0.79, 1.56, 0.80, 1.47, 1.46)

In [665]:
AGq, AGs=8,9
fkhvq = interp1d([1,2,4,6,8,10],[1.01,1.02,1.04,1.06,1.07,1.08])
fkhvs = interp1d([1,2,4,6,8,10],[1.01,1.03,1.05,1.07,1.09,1.12])
fkfvq = interp1d([1,2,4,6,8,10],[1.03,1.06,1.11,1.17,1.23,1.29])
fkfvs = interp1d([1,2,4,6,8,10],[1.04,1.07,1.14,1.21,1.28,1.35])
fkhaq = interp1d([0,2.5,5,10,15],[1.05,1.05,1.09,1.13,1.17])
fkfaq = interp1d([0,2.5,5,10,15],[1.22,1.22,1.27,1.37,1.45])
fkhas = interp1d([0,2.5,5],[1.13,1.13,1.16])
fkfas = interp1d([0,2.5,5],[1.37,1.37,1.4])
khvq, khvs = float(fkhvq(v1q)), float(fkhvs(v1s))
kfvq, kfvs = float(fkfvq(v1q)), float(fkfvs(v1s))
khaq, khas = float(fkhaq(v1q)), float(fkhas(v1s))
kfaq, kfas = float(fkfaq(v1q)), float(fkfas(v1s))
khq, khs = KHbq*khaq*khvq, KHbs*khas*khvs
kfq, kfs = KFbq*kfaq*kfvq, KFbs*kfas*kfvs
sigHq = zm*zhq*zeq*sqrt(2*T_sh1*khq*(uq+1)/(bq*uq*d1q**2))
sigHs = zm*zhs*zes*sqrt(2*T_sh2*khs*(us+1)/(bs*us*d1s**2))

[KHbq, KFbq, KHbs, KFbs], [khvq, kfvq, khvs, kfvs], [khaq, kfaq, khas, kfas], [khq, kfq, khs, kfs], [sigHq, sighq], [sigHs, sighs]

([1.03, 1.08, 1.08, 1.17],
 [1.06, 1.18, 1.03, 1.08],
 [1.10, 1.30, 1.13, 1.37],
 [1.21, 1.66, 1.26, 1.73],
 [251.85, 432.38],
 [380.29, 389.46])

In [666]:
yeq, ybq = ealpq**-1, 1-betq*180/pi/140
fyf = interp1d([17,20,22,25,30,40,50,60,80,100,150],[4.26,4.08,4,3.9,3.8,3.7,3.65,3.62,3.61,3.6,3.6])
zv1q, zv2q = z1q/cos(betq)**3, z2q/cos(betq)**3
yf1q, yf2q = float(fyf(zv1q)), float(fyf(zv2q))
sigF1q = 2*T_sh1*kfq*yeq*ybq*yf1q/bq/d1q/mq/cos(betq)
sigF2q = sigF1q*yf2q/yf1q

yes, ybs = ealps**-1, 1-bets*180/pi/140
fyf = interp1d([17,20,22,25,30,40,50,60,80,100,150],[4.26,4.08,4,3.9,3.8,3.7,3.65,3.62,3.61,3.6,3.6])
zv1s, zv2s = z1s/cos(bets)**3, z2s/cos(bets)**3
yf1s, yf2s = float(fyf(zv1s)), float(fyf(zv2s))
sigF1s = 2*T_sh2*kfs*yes*ybs*yf1s/bs/d1s/ms/cos(bets)
sigF2s = sigF1s*yf2s/yf1s

print("pair 1")
print("y_epsilon = " + str(np.array([yeq])))
print("y_beta = " + str(np.array([ybq])))
print("z_v1 = " + str(np.array([zv1q])))
print("z_v2 = " + str(np.array([zv2q])))
print("y_f1 = " + str(np.array([yf1q])))
print("y_f2 = " + str(np.array([yf2q])))
print("sigF1 = " + str(np.array([sigF1q])) + " <= " + str(np.array([sigf1q])))
print("sigF2 = " + str(np.array([sigF2q])) + " <= " + str(np.array([sigf2q])))

print("\npair 2")
print("y_epsilon = " + str(np.array([yes])))
print("y_beta = " + str(np.array([ybs])))
print("z_v1 = " + str(np.array([zv1s])))
print("z_v2 = " + str(np.array([zv2s])))
print("y_f1 = " + str(np.array([yf1s])))
print("y_f2 = " + str(np.array([yf2s])))
print("sigF1 = " + str(np.array([sigF1s])) + " <= " + str(np.array([sigf1s])))
print("sigF2 = " + str(np.array([sigF2s])) + " <= " + str(np.array([sigf2s])))


pair 1
y_epsilon = [0.62]
y_beta = [0.92]
z_v1 = [43.4]
z_v2 = [122.78]
y_f1 = [3.68]
y_f2 = [3.6]
sigF1 = [41.09] <= [217.57]
sigF2 = [40.17] <= [201.65]

pair 2
y_epsilon = [0.64]
y_beta = [0.91]
z_v1 = [29.13]
z_v2 = [83.08]
y_f1 = [3.82]
y_f2 = [3.61]
sigF1 = [64.37] <= [211.63]
sigF2 = [60.85] <= [196.14]


In [667]:
sigHmaxq = sigHq*sqrt(Kqt)
sigHmaxs = sigHs*sqrt(Kqt)
sigF1maxq = sigF1q*Kqt
sigF2maxq = sigF2q*Kqt
sigF1maxs = sigF1s*Kqt
sigF2maxs = sigF2s*Kqt
[sigHmaxq, sigHmaxs, sighmax], [sigF1maxq, sigF2maxq, sigF1maxs, sigF2maxs, sigfmax]

([267.63, 404.11, 952.00], [46.40, 45.36, 72.69, 68.71, 272.00])

## Shaft design

In [668]:
d12, d21, d22, d31, = d1q, d2q, d1s, d2s
r12, cb12, cq12, hr12 = d12/2, 1, -1, 1
r22, cb22, cq22, hr22 = d22/2, 1, 1, -1

d=30
F11x = 2*T_sh1/d

F12x = np.sign(r12)*cq12*cb12*2*T_sh1/d12; F21x = -F12x
F12y = -np.sign(r12)*2*T_sh1/d12*tan(alpn)/cos(betq); F21y = -F12y
F12z = cq12*cb12*hr12*2*T_sh1/d12*tan(betq); F21z = -F12z

F22x = np.sign(r22)*cq22*cb22*2*T_sh1/d22; F31x = -F22x
F22y = -np.sign(r22)*2*T_sh2/d22*tan(alpn)/cos(bets); F31y = -F22y
F22z = cq22*cb22*hr22*2*T_sh2/d22*tan(bets); F31z = -F22z

alp = 0
F32x = Fr32*cos(alp*pi/180)
F32y = Fr32*sin(alp*pi/180)

print("F11 = " + str(np.array([F11x, 0])))
print("F12 = " + str(np.array([F12x, F12y, F12z])))
print("F21 = " + str(np.array([F21x, F21y, F21z])))
print("F22 = " + str(np.array([F22x, F22y, F22z])))
print("F31 = " + str(np.array([F31x, F31y, F31z])))
print("F32 = " + str(np.array([F32x, F32y])))

F11 = [3094.92    0.  ]
F12 = [-1111.06  -412.12  -218.24]
F21 = [1111.06  412.12  218.24]
F22 = [ 1117.61 -1133.19  -691.82]
F31 = [-1117.61  1133.19   691.82]
F32 = [3741.    0.]


In [669]:
tau1, tau2, tau3 = 20, 25, 30

d1 = (5*T_sh1/tau1)**(1/3)
d2 = (5*T_sh2/tau2)**(1/3)
d3 = (5*T_sh3/tau3)**(1/3)
print("d_shaft min = " + str(np.array([d1,d2,d3])))
d1, d2, d3 = np.ceil(np.array([d1,d2,d3])/5)*5; d1 = 30
print("d_shaft = " + str(np.array([d1,d2,d3])))
ds = np.arange(20,95,5)
bO = np.arange(15,45,2)
fbO = lambda d: bO[np.where(d==ds)][0]
bO1, bO2, bO3 = fbO(d1), fbO(d2), fbO(d3)
print("bearing width = " + str(np.array([bO1, bO2, bO3])))

d_shaft min = [22.64 29.33 38.5 ]
d_shaft = [30. 30. 40.]
bearing width = [19 19 23]


In [670]:
lm12,lm13,lm22,lm32,lm33,k1,k2,k3,hn= 1.4,1.5,1.5,1.5,1.5,5,5,5,15
# lm12,lm13,lm22,lm32,lm33,k1,k2,k3,hn= 1.4,1.4,1.4,1.5,1.2,8,5,10,15
lm12,lm13,lm22,lm32,lm33 = lm12*d1,lm13*d1,lm22*d2,lm32*d3,lm33*d3; lm13,lm22,lm32 = bq, bq, bs
# lm12 = 42
# lm13, lm22, lm32 = 50, 50, 65
# shaft 1
# lm33 = 69.5
l12 = .5*(lm12+bO1)+k3+hn
l13 = .5*(lm13+bO1)+k1+k2
l11 = 2*l13
# shaft 3
l32 = .5*(lm32+bO3)+k1+k2
l31 = 2*l32
l33 = l31 + .5*(lm33+bO3)+k3+hn+0.5
# shaft 2
l22 = l13
l23 = l11+l32+k1+0.5*(bO1+bO3)
l21 = l23+l32

l12 = 62; l13 = 43; l11 = 86
l22 = 45; l23 = 170; l21 = 220
l32 = 51; l31 = 103; l33 = 167
print("lm12, lm33 = " + str(np.array([lm12,lm33])))
print("lm13, lm22 = {0:.2f}, {1:.2f} >= b1 = {2:.2f} ".format(lm13,lm22,bq))
print("lm32 = {0:.2f} >= b2 = {1:.2f} ".format(lm32,bs))
print("l12 = {0:.2f}, l13 = {1:.2f}, l11 = {2:.2f}".format(l12, l13, l11))
print("l32 = {0:.2f}, l31 = {1:.2f}, l33 = {2:.2f}".format(l32, l31, l33))
print("l22 = {0:.2f}, l23 = {1:.2f}, l21 = {2:.2f}".format(l22, l23, l21))

lm12, lm33 = [42. 60.]
lm13, lm22 = 48.00, 48.00 >= b1 = 48.00 
lm32 = 62.00 >= b2 = 62.00 
l12 = 62.00, l13 = 43.00, l11 = 86.00
l32 = 51.00, l31 = 103.00, l33 = 167.00
l22 = 45.00, l23 = 170.00, l21 = 220.00


In [671]:
F1Ax, F1Bx = np.linalg.solve(
    np.array([[1,1],
             [l12,l12+l11]]),
    -np.array([F21x+F11x,(l12+l13)*F21x]))
F1Ay, F1By = np.linalg.solve(
    np.array([[1,1],
             [l12,l12+l11]]),
    -np.array([F21y,(l12+l13)*F21y+d12/2*F21z]))
F2Ax, F2Bx = np.linalg.solve(
    np.array([[1,1],
             [0,l21]]),
    -np.array([F12x+F31x,l22*F12x+l23*F31x]))
F2Ay, F2By = np.linalg.solve(
    np.array([[1,1],
             [0,l21]]),
    -np.array([F12y+F31y,l22*F12y+l23*F31y+d21/2*F12z+d22/2*F31z]))
F3Ax, F3Bx = np.linalg.solve(
    np.array([[1,1],
             [0,l31]]),
    -np.array([F22x+F32x,l32*F22x+l33*F32x]))
F3Ay, F3By = np.linalg.solve(
    np.array([[1,1],
             [0,l31]]),
    -np.array([F22y+F32y,l32*F22y+l33*F32y+d31/2*F22z]))

M1x1 = 0
M1x2 = l12*F1Ay
M1x3m = l12*F1Ay+(l12+l13)*F21y
M1x3p = l12*F1Ay+(l12+l13)*F21y+d12/2*F21z
M1x4 = l12*F1Ay+(l12+l13)*F21y+(l12+l11)*F1By+d12/2*F21z

M1y1 = 0
M1y2 = l12*F1Ax
M1y3 = l12*F1Ax+(l12+l13)*F21x
M1y4 = l12*F1Ax+(l12+l13)*F21x+(l12+l11)*F1Bx

M2x1 = 0
M2x2m = l22*F12y
M2x2p = l22*F12y+d21/2*F12z
M2x3m = l22*F12y+l23*F31y+d21/2*F12z
M2x3p = l22*F12y+l23*F31y+d21/2*F12z+d22/2*F31z
M2x4 =  l22*F12y+l23*F31y+l21*F2By+d21/2*F12z+d22/2*F31z

M2y1 = 0
M2y2 = l22*F12x
M2y3 = l22*F12x+l23*F31x
M2y4 = l22*F12x+l23*F31x+l21*F2Bx

M3x1 = 0
M3x2m = l32*F22y
M3x2p = l32*F22y+d31/2*F22z
M3x3 = l32*F22y+l31*F3By+d31/2*F22z
M3x4 = l32*F22y+l31*F3By+l33*F32y+d31/2*F22z

M3y1 = 0
M3y2 = l32*F22x
M3y3 = l32*F22x+l31*F3Bx
M3y4 = l32*F22x+l31*F3Bx+l33*F32x

M1 = np.array([[M1x1,M1x2,M1x3m,M1x3p,M1x4],[M1y1,M1y2,M1y3,M1y3,M1y4],[T_sh1,T_sh1,T_sh1,0,0]])
M2 = np.array([[M2x1,M2x2m,M2x2p,M2x3m,M2x3p,M2x4],[M2y1,M2y2,M2y2,M2y3,M2y3,M2y4],[0,0,T_sh2,T_sh2,0,0]])
M3 = np.array([[M3x1,M3x2m,M3x2p,M3x3,M3x4],[M3y1,M3y2,M3y2,M3y3,M3y4],[0,0,T_sh3,T_sh3,T_sh3]])

Mf = lambda Mx, My, Tsh: np.sqrt(Mx**2+My**2+.75*Tsh**2)
uf = lambda a, b: a if abs(a)>abs(b) else b # unsigned condition
Mx = np.array([[M1x1,M1x2,uf(M1x3m,M1x3p),M1x4],
              [M2x1,uf(M2x2m,M2x2p),uf(M2x3m,M2x3p),M2x4],
              [M3x1,uf(M3x2m,M3x2p),M3x3,M3x4]])
My = np.array([[M1y1,M1y2,M1y3,M1y4],
              [M2y1,M2y2,M2y3,M2y4],
              [M3y1,M3y2,M3y3,M3y4]])
Ts = np.array([[T_sh1,T_sh1,T_sh1,0],
              [0,T_sh2,T_sh2,0],
              [0,T_sh3,T_sh3,T_sh3]])

Me1 = Mf(M1[0,:],M1[1,:],M1[2,:])
Me2 = Mf(M2[0,:],M2[1,:],M2[2,:])
Me3 = Mf(M3[0,:],M3[1,:],M3[2,:])

sigf1 = interp1d([30,50,100],[58,48,45])
sigf2 = interp1d([30,50,100],[63,50,48])
sigf3 = interp1d([30,50,100],[67,55,50])
d1s = (10*Me1/sigf1(d1))**(1/3)*1.04
d2s = (10*Me2/sigf1(d2))**(1/3)*1.1
d3s = (10*Me3/sigf1(d3))**(1/3)*1.04

print("F1A = " + str(np.array([F1Ax, F1Ay])) + "\t, F1B = " + str(np.array([F1Bx, F1By])))
print("F2A = " + str(np.array([F2Ax, F2Ay])) + "\t, F2B = " + str(np.array([F2Bx, F2By])))
print("F3A = " + str(np.array([F3Ax, F3Ay])) + "\t, F3B = " + str(np.array([F3Bx, F3By])))
print("M1 = \n" + str(np.concatenate((M1, [Me1]))))
print("M2 = \n" + str(np.concatenate((M2, [Me2]))))
print("M3 = \n" + str(np.concatenate((M3, [Me3]))))
print("d1 at 1, 2, 3-, 3+, 4 =      " + str(d1s))
print("d2 at 1, 2-, 2+, 3-, 3+, 4 = " + str(d2s))
print("d3 at 1, 2-, 2+, 3, 4 =      " + str(d3s))

F1A = [-5881.66  -100.03]	, F1B = [1675.69 -312.09]
F2A = [1137.8    83.63]	, F2B = [1090.87 -804.7 ]
F3A = [1760.27 -223.57]	, F3B = [-6618.88  1356.76]
M1 = 
[[      0.     -6201.76   37070.68   46189.33       0.  ]
 [      0.   -364663.03 -248002.17 -248002.17       0.  ]
 [  46423.73   46423.73   46423.73       0.         0.  ]
 [  40204.13  366925.01  253960.    252266.78       0.  ]]
M2 = 
[[      0.    -18545.33  -44344.45  148297.9   177034.87       0.  ]
 [      0.    -49997.51  -49997.51 -239990.94 -239990.94       0.  ]
 [      0.         0.    126093.3   126093.3        0.         0.  ]
 [      0.     53326.17  128026.64  302510.43  298223.07       0.  ]]
M3 = 
[[      0.    -57792.7  -139746.28       0.         0.  ]
 [      0.     56998.03   56998.03 -624746.19       0.  ]
 [      0.         0.    342486.86  342486.86  342486.86]
 [      0.     81171.25  332792.33  691578.44  296602.32]]
d1 at 1, 2, 3-, 3+, 4 =      [19.83 41.44 36.66 36.57  0.  ]
d2 at 1, 2-, 2+, 3-, 3+,

In [672]:
# b1k, h1k, t11k, t12k, l1k, r1k = 10, 8, 5, 3.3, 50, 0.3
d1s = np.array([30,35,42,35]); D1s = np.hstack(([d1s[0]], [34], d1s[1:]))
# d2s = np.array([50,57,57,50]); D2s = np.hstack((d2s[:2], [60], d2s[2:]))
d2s = np.array([45,54,54,45]); D2s = np.hstack((d2s[:2], [60], d2s[2:]))
d3s = np.array([60,69,60,52]); D3s = np.hstack(([d3s[0]], [84], d3s[1:]))
ds = np.array([d1s, d2s, d3s]); Ds = np.array([D1s, D2s, D3s])
bck, hck, tc1k, tc2k, lck, rck = 10, 8, 5, 3.3, 32, 0.4
b1k, h1k, t11k, t12k, l1k, r1k = 12, 8, 5, 3.3, 32, 0.3
# b2k, h2k, t21k, ct22k, l2k, r2k = 12, 8, 5, 3.3, 40, 0.3
b21k, h21k, t211k, t212k, l21k, r21k = 16, 10, 6, 4.3, 32, 0.4
b22k, h22k, t221k, t222k, l22k, r22k = 16, 10, 6, 4.3, 45, 0.4
# b2k, h2k, t21k, ct22k, l2k, r2k = 14, 9, 5.5, 3.8, 36, 0.3
b3k, h3k, t31k, t32k, l3k, r3k = 18, 11, 7, 4.4, 45, 0.4
# b3k, h3k, t31k, t32k, l3k, r3k = 20, 12, 7.5, 4.9, 40, 0.3 #14, 9, 5.5, 3.8, 65, 0.3
bsk, hsk, ts1k, cts2k, lsk, rsk = 16, 10, 6, 4.3, 45, 0.4

sigf = lambda T, d, l, h, t: 2*T/(d*l*(h-t))
tauf = lambda T, d, l, b: 2*T/d/l/b

sigc = sigf(T_sh1, d1s[0], lck, hck, tc1k)
sigd1 = sigf(T_sh1, d1s[2], l1k, h1k, t11k)
sigd21 = sigf(T_sh2, d2s[1], l21k, h21k, t211k)
sigd22 = sigf(T_sh2, d2s[2], l22k, h22k, t221k)
sigd3 = sigf(.75*T_sh3, d3s[1], l3k, h3k, t31k)
sigs = sigf(.75*T_sh3, d3s[-1], lsk, hsk, ts1k)

tauc = tauf(T_sh1, d1s[0], lck, bck)
tau1 = tauf(T_sh1, d1s[2], l1k, b1k)
tau21 = tauf(T_sh2, d2s[1], l21k, b21k)
tau22 = tauf(T_sh2, d2s[2], l22k, b22k)
tau3 = tauf(T_sh3, d3s[1], l3k, b3k)
taus = tauf(T_sh3, d3s[-1], lsk, bsk)

sigpm, taupm = 100, 60
print("sigd = " + str(np.array([sigc, sigd1, sigd21, sigd22, sigd3, sigs])) + " <= " + str(sigpm))
print("tau = " + str(np.array([tauc, tau1, tau21, tau22, tau3, taus])) + " <= " + str(1/3*taupm))

sigd = [32.24 23.03 36.49 25.95 41.36 54.89] <= 100
tau = [ 9.67  5.76  9.12  6.49 12.26 18.3 ] <= 20.0


In [673]:
# Ktau2fsigb = 600
sigb = 585
siginv = .436*sigb
tauinv = .58*siginv

psisig, psitau, Kx, Ky = 0.05, 0, 1.2, 1

# Ksig_epsig = np.where(np.array([d1s, d2s, d3s])>50,2.52,2.06)
# Ktau_eptau = np.where(np.array([d1s, d2s, d3s])>=50,2.03,1.64)
Ksig1, Ktau1 = 1.76, 1.54
Ksig2f = interp1d([0,.02,.06,.1,.16],[3,2.5,1.85,1.6,1.49])
Ktau2f = interp1d([0,.02,.06,.1,.16],[2.1,1.8,1.4,1.25,1.18])
rs = np.array([[.5,.5,.5,.5],
             [.5,.5,.5,.5],
             [.5,.5,1.5,.5]])
rds = rs/ds
# rdf = lambda d: np.hstack(abs(Ds[:,:-1]-Ds[:,1:])/2/d[:,1:])
# rds = rdf(Ds).reshape(3,4)
# rds[0,-1] = rds[1,0] = rds[1,-1] = rds[2,0] = rds[2,2] = .02; rds = np.hstack(rds);
Ksigs, Ktaus = Ksig2f(rds).reshape(3,4), Ktau2f(rds).reshape(3,4)
esigf = interp1d([15,20,30,40,50,70,80,100],[.95,.92,.88,.85,.81,.76,.73,.70])
etauf = interp1d([15,20,30,40,50,70,80,100],[.92,.89,.81,.78,.76,.73,.71,.70])
esigs, etaus = esigf([d1s, d2s, d3s]), etauf([d1s, d2s, d3s])
print("r/d = \n" + str(rds.reshape(3,4)))
print("Ksig = \n" + str(Ksigs))
print("Ktau = \n" + str(Ktaus))
print("esig = \n" + str(esigs))
print("etau = \n" + str(etaus))

r/d = 
[[0.02 0.01 0.01 0.01]
 [0.01 0.01 0.01 0.01]
 [0.01 0.01 0.03 0.01]]
Ksig = 
[[2.58 2.64 2.7  2.64]
 [2.72 2.77 2.77 2.72]
 [2.79 2.82 2.42 2.76]]
Ktau = 
[[1.85 1.89 1.92 1.89]
 [1.93 1.96 1.96 1.93]
 [1.98 1.99 1.75 1.96]]
esig = 
[[0.88 0.86 0.84 0.86]
 [0.83 0.8  0.8  0.83]
 [0.79 0.76 0.79 0.81]]
etau = 
[[0.81 0.8  0.78 0.8 ]
 [0.77 0.75 0.75 0.77]
 [0.74 0.73 0.74 0.76]]


In [674]:
W1f = lambda d, b, t: pi*d**3/32 - b*t*(d-t)**2/2/d # 1 keyseat
W2f = lambda d, b, t: pi*d**3/32 - b*t*(d-t)**2/d # 2 keyseats
Wo1f = lambda d, b, t: pi*d**3/16 - b*t*(d-t)**2/2/d # 1 keyseat
Wo2f = lambda d, b, t: pi*d**3/16 - b*t*(d-t)**2/d # 2 keyseats
b1k = np.array([bck,0,b1k,0])
b2k = np.array([0,b21k,b22k,0])
b3k = np.array([0,b3k,0,bsk])
t11k = np.array([tc1k,0,t11k,0])
t21k = np.array([0,t211k,t221k,0])
t31k = np.array([0,t31k,0,ts1k])
Ws = np.array([W1f(d1s, b1k, t11k), W2f(d2s, b2k, t211k), W1f(d3s, b3k, t31k)])
Wos = np.array([Wo1f(d1s, b1k, t11k), Wo2f(d2s, b2k, t211k), Wo1f(d3s, b3k, t31k)])
Ws
print("W = \n" + str(Ws))
print("Wo = \n" + str(Wos))

sigms = 0
sigas = np.sqrt(Mx**2+My**2)/Ws
print("sigm = 0; siga = \n" + str(sigas))
taums = tauas = Ts/2/Wos
print("taum = taua = \n" + str(tauas))

W = 
[[ 2129.89  4209.24  6295.72  4209.24]
 [ 8946.18 11362.99 11362.99  8946.18]
 [21205.75 28741.56 21205.75 11850.93]]
Wo = 
[[ 4780.6   8418.49 13569.29  8418.49]
 [17892.35 26821.98 26821.98 17892.35]
 [42411.5  60992.85 42411.5  25655.09]]
sigm = 0; siga = 
[[ 0.   86.65 40.07  0.  ]
 [ 0.    5.88 26.25  0.  ]
 [ 0.    5.25 29.46  0.  ]]
taum = taua = 
[[4.86 2.76 1.71 0.  ]
 [0.   2.35 2.35 0.  ]
 [0.   2.81 4.04 6.67]]


In [675]:
import warnings
warnings.filterwarnings('ignore')

Ksigds = (Ksigs/esigs + Kx - 1)/Ky
Ktauds = (Ktaus/etaus + Kx - 1)/Ky

ssig = siginv/(Ksigds*sigas+psisig*sigms)
ssig = np.where(ssig>1e10,np.inf,ssig)
stau = tauinv/(Ktauds*tauas+psitau*taums)
stau = np.where(stau>1e10,np.inf,stau)

s = ssig*stau/np.sqrt(ssig**2+stau**2)
print("Ksigd = \n" + str(Ksigds))
print("Ktaud = \n" + str(Ktauds))
print("ssig = \n" + str(ssig[:,:]))
print("stau = \n" + str(stau))
print("s = \n" + str(s))

Ksigd = 
[[3.14 3.26 3.41 3.26]
 [3.48 3.66 3.66 3.48]
 [3.76 3.9  3.28 3.63]]
Ktaud = 
[[2.48 2.57 2.68 2.57]
 [2.71 2.8  2.8  2.71]
 [2.85 2.92 2.55 2.78]]
ssig = 
[[  inf  0.9   1.87   inf]
 [  inf 11.85  2.65   inf]
 [  inf 12.46  2.64   inf]]
stau = 
[[12.27 20.86 32.32   inf]
 [  inf 22.47 22.47   inf]
 [  inf 18.03 14.37  7.96]]
s = 
[[  nan  0.9   1.86   nan]
 [  nan 10.48  2.64   nan]
 [  nan 10.25  2.6    nan]]


In [676]:
sige = np.sqrt((np.sqrt(Mx**2+My**2)*10/d**3)**2 + 3*(Ts/0.2/d**3)**2)
sige

array([[ 14.89, 135.9 ,  94.61,   0.  ],
       [  0.  ,  47.42, 117.62,   0.  ],
       [  0.  , 123.26, 256.14, 109.85]])

## bearing design

In [677]:
Fxs = np.array([F1Ax, F1Bx, F2Ax, F2Bx, F3Ax, F3Bx])/1e3
Fys = np.array([F1Ay, F1By, F2Ay, F2By, F3Ay, F3By])/1e3
Fzs = np.array([F21z, 0, F12z+F31z, 0, F22z, 0])/1e3
Frs = np.sqrt(Fxs**2+Fys**2)
Fas = np.abs(Fzs)
Fars = Fas/Frs
print("shaft 1: d1 = {0:.2f}, b1 = {1:.2f}".format(d1s[1], bO1))
print("shaft 2: d2 = {0:.2f}, b2 = {1:.2f}".format(d2s[0], bO2))
print("shaft 3: d3 = {0:.2f}, b3 = {1:.2f}".format(d3s[0], bO3))
print("Fx = " + str(Fxs))
print("Fy = " + str(Fys))
print("Fz = " + str(Fzs))
print("Fr = " + str(Frs))
print("Fa = " + str(Fas))
print("Fa/Fr = " + str(Fars))

shaft 1: d1 = 35.00, b1 = 19.00
shaft 2: d2 = 45.00, b2 = 19.00
shaft 3: d3 = 60.00, b3 = 23.00
Fx = [-5.88  1.68  1.14  1.09  1.76 -6.62]
Fy = [-0.1  -0.31  0.08 -0.8  -0.22  1.36]
Fz = [ 0.22  0.    0.47  0.   -0.69  0.  ]
Fr = [5.88 1.7  1.14 1.36 1.77 6.76]
Fa = [0.22 0.   0.47 0.   0.69 0.  ]
Fa/Fr = [0.04 0.   0.42 0.   0.39 0.  ]


In [686]:
ns = np.repeat([n_sh1, n_sh2, n_sh3],2)
Ds = np.array([72,85,120])
ds = np.array([d1s[1],d2s[0],d3s[0]])
Cs = np.repeat([29.1, 35.8, 66.3],2)
C0s = np.repeat([19, 26, 54],2)
Pus = np.repeat([0.85, 1.12, 1.93],2)
As = np.repeat([.00674, .014, .0478],2)
krs = np.repeat([.095, .095, .095],2)
Rs = np.array([.88,.88,.88])
dms = np.repeat((Ds+ds)/2,2)
dms, ns*dms

(array([53.5, 53.5, 65. , 65. , 90. , 90. ]),
 array([77842.5 , 77842.5 , 33437.31, 33437.31, 16368.75, 16368.75]))

In [687]:
FRs = Frs[[0,2,4]]
FAs = Fas[[0,2,4]]
FARs = Fars[[0,2,4]]
FAs, Rs*(Frs[[1,3,5]]-Frs[[0,2,4]])

(array([5.18, 1.  , 1.56]), array([-3.68,  0.19,  4.38]))

In [680]:
Fas = np.array([Rs[0]*FRs[0], Rs[0]*FRs[0]+FAs[0],
       Rs[1]*FRs[1], Rs[1]*FRs[1]+FAs[1],
       Rs[2]*FRs[2], Rs[2]*FRs[2]-FAs[2]])
Frs, Fas

(array([5.88, 1.7 , 1.14, 1.36, 1.77, 6.76]),
 array([5.18, 5.39, 1.  , 1.48, 1.56, 0.87]))

In [681]:
nu = 13
Frms = krs * (nu*ns/1000)**(2/3) * (dms/100)**2
Frms

array([0.19, 0.19, 0.14, 0.14, 0.14, 0.14])

In [682]:
e, X, Y1, Y2, Y0 = 1.14, .57, .55, .93, .52
Ps = np.where(Fars<=e,Frs+Y1*Fas,X*Frs+Y2*Fas)*1.05
P0s = Frs+Y0*Fas
etac = .5
np.array([Frs,Fas,Fas/Frs, Ps]).T

array([[5.88, 5.18, 0.88, 9.17],
       [1.7 , 5.39, 3.17, 4.91],
       [1.14, 1.  , 0.88, 1.78],
       [1.36, 1.48, 1.09, 2.28],
       [1.77, 1.56, 0.88, 2.76],
       [6.76, 0.87, 0.13, 7.6 ]])

In [683]:
a1 = 1
np.array([ns, ns*dms]).T

array([[ 1455.  , 77842.5 ],
       [ 1455.  , 77842.5 ],
       [  514.42, 33437.31],
       [  514.42, 33437.31],
       [  181.88, 16368.75],
       [  181.88, 16368.75]])

In [684]:
factors = etac*Pus/Ps
nu1s = np.repeat([15, 30, 48],2)
ks = nu/nu1s
askfs = np.array([15,10,1.1,.9,.22,.2])
np.array([Pus,Ps,factors,ks,askfs]).T

array([[ 0.85,  9.17,  0.05,  0.87, 15.  ],
       [ 0.85,  4.91,  0.09,  0.87, 10.  ],
       [ 1.12,  1.78,  0.32,  0.43,  1.1 ],
       [ 1.12,  2.28,  0.25,  0.43,  0.9 ],
       [ 1.93,  2.76,  0.35,  0.27,  0.22],
       [ 1.93,  7.6 ,  0.13,  0.27,  0.2 ]])

In [685]:
Lnmhs = 1e6/60/ns*a1*askfs*(Cs/Ps)**3
np.array([ns,askfs,Cs,Ps,Lnmhs]).T

array([[  1455.  ,     15.  ,     29.1 ,      9.17,   5497.91],
       [  1455.  ,     10.  ,     29.1 ,      4.91,  23915.53],
       [   514.42,      1.1 ,     35.8 ,      1.78, 291071.18],
       [   514.42,      0.9 ,     35.8 ,      2.28, 113384.06],
       [   181.88,      0.22,     66.3 ,      2.76, 277974.62],
       [   181.88,      0.2 ,     66.3 ,      7.6 ,  12184.16]])